In [1]:
import json
import pandas as pd
import requests
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#pd.read_json('data/business.json', orient='records')
df_b = pd.read_json('data/business.json', lines=True)

In [3]:
def iceCreamFinder(categories):
    '''Finds the businesses that have the ice cream tag'''
    if categories:
        return (('Ice Cream & Frozen Yogurt') in categories.split(', '))
    return False

In [4]:
ic_business = df_b['categories'].apply(iceCreamFinder)
ic_df = df_b[ic_business & (df_b['is_open']!=0)]

In [5]:
len(ic_df)

1957

In [6]:
ic_df['attributes'][125]

{'RestaurantsPriceRange2': '1', 'RestaurantsTakeOut': 'True'}

In [7]:
ic_df['state'].unique()

array(['ON', 'NC', 'PA', 'AZ', 'QC', 'AB', 'NV', 'OH', 'IL', 'WI', 'BAS',
       'SC'], dtype=object)

In [8]:
pd.Series(ic_df['name']).value_counts()[0:50]

Dairy Queen                               138
Sonic Drive-In                             88
Cold Stone Creamery                        49
Baskin-Robbins                             33
Culver's                                   33
Baskin Robbins                             27
Menchie's Frozen Yogurt                    22
Nekter Juice Bar                           21
DQ Grill & Chill Restaurant                20
Marble Slab Creamery                       19
Rita's Italian Ice                         19
La Diperie                                 18
Freddy's Frozen Custard & Steakburgers     17
Yogurtland                                 14
Ben & Jerry's                              13
Sweet Frog Premium Frozen Yogurt           13
Chocolats Favoris                          12
Sincerely Yogurt                            9
TCBY                                        9
Menchie's                                   9
Hwy 55 Burgers Shakes & Fries               8
Chocolate Shoppe Ice Cream        

In [9]:
city='San Diego'
def cityIceCream(city):
    api_key='Nn6XVo7jwIf3l9Qu1QvTwbp7aBH3fA0kOD61VxLOywsFlqD_AGqRahhAgi1aGq1mFWU5N04D-qyQmj_Em66dWMoE3VB9bQ_95OF3ZU0lXollDxqBbJYwNHa5bz9fXnYx'
    headers = {'Authorization': 'Bearer %s' % api_key}
    url='https://api.yelp.com/v3/businesses/search'
    
    params={'location':city, 'categories': 'icecream', 'radius':40000, 'limit':50, 'offset':0}
    req=requests.get(url, params=params, headers=headers)
    print('The status code for Part 1 {} is {}'.format(city, req.status_code))
    output_json = json.loads(req.text)
    
    params2={'location':city, 'categories': 'icecream', 'radius':40000, 'limit':50, 'offset':50}
    req2=requests.get(url, params=params2, headers=headers)
    print('The status code for Part 2 {} is {}'.format(city, req.status_code))
    output_json2 = json.loads(req2.text)
    return pd.concat([pd.DataFrame(output_json['businesses']), pd.DataFrame(output_json2['businesses'])], sort=False)
                      
                      

In [10]:
dummy = pd.DataFrame()
city_list=['San Diego', 'Los Angeles']
for city in city_list:
    dummy = pd.concat([dummy, cityIceCream(city)], sort=False).drop_duplicates(subset='id').reset_index(drop=True)
    ice_cream_df = pd.merge(dummy, dummy['location'].apply(pd.Series)[['city', 'zip_code', 'state']], left_index=True, right_index=True)
    
    

The status code for Part 1 San Diego is 200
The status code for Part 2 San Diego is 200
The status code for Part 1 Los Angeles is 200
The status code for Part 2 Los Angeles is 200


In [11]:
ice_cream_df['name'].value_counts()

Salt & Straw                      5
Yogurtland Los Angeles            5
Pressed Juicery                   4
Hammond's Gourmet Ice Cream       3
Mateo's Ice Cream & Fruit Bars    3
                                 ..
Paradise Yogurt                   1
Cauldron Ice Cream                1
Dr. Freeze Amazing Desserts       1
CVT Soft Serve                    1
Stella Jean's Ice Cream           1
Name: name, Length: 165, dtype: int64

In [12]:
ice_cream_df['price'].value_counts()

$     128
$$     59
Name: price, dtype: int64

In [13]:
ice_cream_df.groupby('city')['price'].value_counts()

city            price
Beverly Hills   $$        1
Burbank         $         1
Coronado        $         2
Culver City     $         2
Glendale        $         1
Hollywood       $         2
La Jolla        $         1
Los Angeles     $        52
                $$       35
National City   $         2
San Diego       $        64
                $$       22
West Hollywood  $         1
                $$        1
Name: price, dtype: int64

In [14]:
ice_cream_df[ice_cream_df['name']=='Baskin-Robbins']

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance,city,zip_code,state
93,Zs-VPPv3RTaWJ7FQdAT_-w,baskin-robbins-san-diego-20,Baskin-Robbins,https://s3-media1.fl.yelpcdn.com/bphoto/l7prpW...,False,https://www.yelp.com/biz/baskin-robbins-san-di...,51,"[{'alias': 'icecream', 'title': 'Ice Cream & F...",3.5,"{'latitude': 32.8195147379154, 'longitude': -1...",[delivery],$,"{'address1': '5575 Balboa Ave', 'address2': ''...",+18582925600,(858) 292-5600,4067.933763,San Diego,92111,CA


In [15]:
br = pd.concat([df_b[(df_b['name']=='Baskin Robbins')], df_b[(df_b['name']=='Baskin-Robbins')]], sort=False)

In [16]:
br=df_b[(df_b.name=='Baskin Robbins') | (df_b.name=='Baskin-Robbins') | (df_b.name=='Baskin-Robbins Ice Cream')]

In [17]:
br

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
564,221yaXPE1zmJqDvBVM__tw,Baskin-Robbins,"15530 W Roosevelt St, Ste E105",Goodyear,AZ,85338,33.457571,-112.393151,2.5,26,1,"{'BikeParking': 'True', 'RestaurantsGoodForGro...","American (Traditional), Ice Cream & Frozen Yog...",None
1352,mLI9Jf7byDnT6WDXsnlW5Q,Baskin-Robbins Ice Cream,300 Borough Drive,Scarborough,ON,M1P 4P5,43.776756,-79.259008,4.5,3,1,"{'WiFi': ''no'', 'RestaurantsPriceRange2': '1'}","Ice Cream & Frozen Yogurt, Juice Bars & Smooth...",None
16205,NoRU47_av9qspawxsy9c_A,Baskin Robbins,4160 Baldwin Street,Whitby,ON,L1R 3H8,43.918822,-78.959957,4.0,4,1,"{'RestaurantsPriceRange2': '2', 'BikeParking':...","Food, Ice Cream & Frozen Yogurt",None
23976,K8rtht6Yqu4EbZfXKnExfw,Baskin-Robbins,"4406 E Main Street, Ste 101",Mesa,AZ,85205,33.416137,-111.735743,3.0,17,1,"{'BusinessParking': '{'garage': False, 'street...","Food, Ice Cream & Frozen Yogurt",None
26623,td4s08UNrPK4OfynAF1VSw,Baskin-Robbins,"4326 W Bell Rd, Ste 1",Glendale,AZ,85308,33.639735,-112.153631,3.0,29,1,"{'BusinessAcceptsCreditCards': 'True', 'Caters...","Ice Cream & Frozen Yogurt, Food",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179405,s2xpXws-b-bSIpWIssvH8w,Baskin-Robbins,4547 E Cactus Rd,Phoenix,AZ,85032,33.598644,-111.985318,3.0,25,1,"{'BusinessParking': '{'garage': False, 'street...","Ice Cream & Frozen Yogurt, Food","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
181205,jWe6JS8hnECIHWt9tUsnfw,Baskin-Robbins,1715 E Guadalupe Rd,Tempe,AZ,85283,33.363422,-111.912603,4.0,23,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Food, Ice Cream & Frozen Yogurt","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
185777,DhvkVOnDDWwL-AXYtxIFgQ,Baskin Robbins,6212 Yonge Street,Toronto,ON,M2M 3X4,43.794603,-79.419204,4.0,5,1,"{'RestaurantsTakeOut': 'True', 'WiFi': 'u'no''...","Desserts, Food, Custom Cakes, Ice Cream & Froz...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
187654,NfnidMkIIb4OcDpP6jrsgw,Baskin-Robbins,"4025 E Chandler Blvd, Ste 64",Phoenix,AZ,85048,33.304461,-111.994519,3.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'Busine...","Food, Ice Cream & Frozen Yogurt",None


In [18]:
city='San Diego'
def iceCreamStore(city, search):
    api_key='Nn6XVo7jwIf3l9Qu1QvTwbp7aBH3fA0kOD61VxLOywsFlqD_AGqRahhAgi1aGq1mFWU5N04D-qyQmj_Em66dWMoE3VB9bQ_95OF3ZU0lXollDxqBbJYwNHa5bz9fXnYx'
    headers = {'Authorization': 'Bearer %s' % api_key}
    url='https://api.yelp.com/v3/businesses/search'
    
    data = []
    
    for offset in range(0, 1000, 50):
        params={'term': search, 'location':city, 'categories': 'icecream', 'radius':40000, 'limit':50, 'offset':0}
        req=requests.get(url, params=params, headers=headers)
        print('The status code for {} {} is {} for {}'.format(offset, city, req.status_code, search))
        output_json = json.loads(req.text)
        df = pd.DataFrame(output_json['businesses'])
        data.append(df)
                       
    return pd.concat(data, sort=False).drop_duplicates(subset='id').reset_index(drop=True)

In [19]:
br2 = iceCreamStore('San Diego', 'baskin robbin')

The status code for 0 San Diego is 200 for baskin robbin
The status code for 50 San Diego is 200 for baskin robbin
The status code for 100 San Diego is 200 for baskin robbin
The status code for 150 San Diego is 200 for baskin robbin
The status code for 200 San Diego is 200 for baskin robbin
The status code for 250 San Diego is 200 for baskin robbin
The status code for 300 San Diego is 200 for baskin robbin
The status code for 350 San Diego is 200 for baskin robbin
The status code for 400 San Diego is 200 for baskin robbin
The status code for 450 San Diego is 200 for baskin robbin
The status code for 500 San Diego is 200 for baskin robbin
The status code for 550 San Diego is 200 for baskin robbin
The status code for 600 San Diego is 200 for baskin robbin
The status code for 650 San Diego is 200 for baskin robbin
The status code for 700 San Diego is 200 for baskin robbin
The status code for 750 San Diego is 200 for baskin robbin
The status code for 800 San Diego is 200 for baskin robbin


In [25]:
br2[(br2['name']=='Baskin Robbins')|(br2['name']=='Baskin-Robbins')|(br2.name=='Baskin-Robbins Ice Cream')]

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,9bfFavK0y7mgavSGwtIYaw,baskin-robbins-san-diego-22,Baskin Robbins,https://s3-media4.fl.yelpcdn.com/bphoto/4Rtyxe...,False,https://www.yelp.com/biz/baskin-robbins-san-di...,36,"[{'alias': 'icecream', 'title': 'Ice Cream & F...",3.5,"{'latitude': 32.80947, 'longitude': -117.11516}",[delivery],$,"{'address1': '3737 Murphy Canyon Rd', 'address...",+18584960707,(858) 496-0707,4190.323771
1,Zs-VPPv3RTaWJ7FQdAT_-w,baskin-robbins-san-diego-20,Baskin-Robbins,https://s3-media1.fl.yelpcdn.com/bphoto/l7prpW...,False,https://www.yelp.com/biz/baskin-robbins-san-di...,51,"[{'alias': 'icecream', 'title': 'Ice Cream & F...",3.5,"{'latitude': 32.8195147379154, 'longitude': -1...",[delivery],$,"{'address1': '5575 Balboa Ave', 'address2': ''...",+18582925600,(858) 292-5600,4067.933763
2,bYecS4GJDKZ7LnYs3lH4Dw,baskin-robbins-san-diego-17,Baskin Robbins,https://s3-media1.fl.yelpcdn.com/bphoto/qIXVNc...,False,https://www.yelp.com/biz/baskin-robbins-san-di...,78,"[{'alias': 'icecream', 'title': 'Ice Cream & F...",2.5,"{'latitude': 32.75376, 'longitude': -117.22245}",[delivery],$,"{'address1': '4015 W Point Loma Blvd', 'addres...",+16192222888,(619) 222-2888,7602.243255
3,QF3ywPsoLEgLWtO1iuQClQ,baskin-robbins-san-diego-30,Baskin-Robbins,https://s3-media3.fl.yelpcdn.com/bphoto/3C_Dx5...,False,https://www.yelp.com/biz/baskin-robbins-san-di...,10,"[{'alias': 'icecream', 'title': 'Ice Cream & F...",4.0,"{'latitude': 32.7557, 'longitude': -117.13346}",[delivery],NaN,"{'address1': '2810 El Cajon Blvd', 'address2':...",+16192558402,(619) 255-8402,4324.419533
4,sRrMEy9ccP2Zm6JF8p8gQQ,baskin-robbins-san-diego-7,Baskin Robbins,https://s3-media1.fl.yelpcdn.com/bphoto/BLeCVp...,True,https://www.yelp.com/biz/baskin-robbins-san-di...,58,"[{'alias': 'icecream', 'title': 'Ice Cream & F...",3.0,"{'latitude': 32.868055, 'longitude': -117.2151...",[],$,"{'address1': '8650 Genesee Ave', 'address2': '...",+18585529080,(858) 552-9080,10332.344819
5,wYG0bUIz-ulnRybSnMta5Q,baskin-robbins-san-diego-28,Baskin-Robbins,https://s3-media1.fl.yelpcdn.com/bphoto/aq5QoH...,False,https://www.yelp.com/biz/baskin-robbins-san-di...,96,"[{'alias': 'icecream', 'title': 'Ice Cream & F...",3.5,"{'latitude': 32.91349, 'longitude': -117.14666}",[delivery],$,"{'address1': '8250 Mira Mesa Blvd', 'address2'...",+18585664031,(858) 566-4031,13729.123205
6,7vxWYrroWg2K_oxYAXyiRQ,baskin-robbins-san-diego-31,Baskin-Robbins,https://s3-media2.fl.yelpcdn.com/bphoto/CDun6e...,False,https://www.yelp.com/biz/baskin-robbins-san-di...,12,"[{'alias': 'icecream', 'title': 'Ice Cream & F...",4.0,"{'latitude': 32.7964909, 'longitude': -117.193...",[delivery],NaN,"{'address1': '3077-A Clairemont Dr', 'address2...",+16197982073,(619) 798-2073,3630.722316
7,P6r8Edutsu5WaRV6UrpLQA,baskin-robbins-san-diego-3,Baskin Robbins,https://s3-media3.fl.yelpcdn.com/bphoto/_7h12n...,False,https://www.yelp.com/biz/baskin-robbins-san-di...,68,"[{'alias': 'icecream', 'title': 'Ice Cream & F...",3.5,"{'latitude': 32.9395261893493, 'longitude': -1...",[delivery],$,"{'address1': '3840 Valley Centre Dr', 'address...",+18583504531,(858) 350-4531,18071.886755
8,fO8dE95ewdceE1b9e1M_mw,baskin-robbins-san-diego-5,Baskin Robbins,https://s3-media2.fl.yelpcdn.com/bphoto/sY_GzN...,False,https://www.yelp.com/biz/baskin-robbins-san-di...,66,"[{'alias': 'icecream', 'title': 'Ice Cream & F...",3.0,"{'latitude': 32.9784212496217, 'longitude': -1...",[delivery],$,"{'address1': '11495A Carmel Mountain Rd', 'add...",+18585920031,(858) 592-0031,21900.264356
9,wVCRxzDr7p4Ei6NRraosFA,baskin-robbins-la-mesa,Baskin Robbins,https://s3-media3.fl.yelpcdn.com/bphoto/SpbRHm...,False,https://www.yelp.com/biz/baskin-robbins-la-mes...,45,"[{'alias': 'icecream', 'title': 'Ice Cream & F...",3.5,"{'latitude': 32.7707901, 'longitude': -117.007...",[delivery],$$,"{'address1': '8807 La Mesa Blvd', 'address2': ...",+16194627215,(619)

In [28]:
bid = '9bfFavK0y7mgavSGwtIYaw'
api_key='Nn6XVo7jwIf3l9Qu1QvTwbp7aBH3fA0kOD61VxLOywsFlqD_AGqRahhAgi1aGq1mFWU5N04D-qyQmj_Em66dWMoE3VB9bQ_95OF3ZU0lXollDxqBbJYwNHa5bz9fXnYx'
url = "https://api.yelp.com/v3/businesses/{}/reviews".format(bid)
headers = {'Authorization': 'Bearer %s' % api_key}
req = requests.get(url, headers=headers)
print('the status code is {}'.format(req.status_code))
review = json.loads(req.text)

the status code is 200


In [33]:
reviews=pd.DataFrame(review['reviews'])

In [34]:
reviews

,id,url,text,rating,time_created,user
0,fSiv2YmlkOjiYWDeewNIJA,https://www.yelp.com/biz/baskin-robbins-san-di...,BR has my favorite ice cream in the world. Cho...,5,2019-09-17 21:57:09,"{'id': 'ETLvNYJ88kq4jhZbRnOJHw', 'profile_url'..."
1,6hEjm-os-8bCIcV1-aUq3w,https://www.yelp.com/biz/baskin-robbins-san-di...,We like coming to this place. We got our sons ...,5,2019-04-20 19:20:42,"{'id': '5bRWowfUqe1IFlk33TMwsA', 'profile_url'..."
2,FwlcRtP7MsuU3aSqEkgCCA,https://www.yelp.com/biz/baskin-robbins-san-di...,$1.50 scoop night!!! Time to treat the family....,3,2018-03-31 21:24:32,"{'id': 'EsX2orTpfIv0kOBMmKYIrw', 'profile_url'..."
